
    
#   <div style="text-align: center;"> Méthodes de Monte Carlo (2023-2024)  </div>
#   <div style="text-align: center;">TP5: Réduction de la variance
  </div>     

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

# Objectif

On se propose d'implémenter des techniques de simulation telles que l'échantillonnage préférentiel et les variables antithétiques qui visent à réduire la variance de l'estimateur de Monte-Carlo.

# Partie 2: Variables antithétiques

Cette méthode consiste à utiliser des variables aléatoires corrélées négativement afin de réduire la variance de l'estimateur. Plus précisément, au lieu de générer une seule suite de variables aléatoires, on en génère deux qui sont corrélées négativement, c'est-à-dire que lorsque l'une prend une valeur élevée, l'autre prend une valeur faible et vice versa.

L'estimateur d'intérêt est ensuite calculé en utilisant la moyenne des observations des deux suites de variables aléatoires. Ainsi, l'estimateur obtenu est plus précis que celui obtenu avec la méthode Monte-Carlo classique; Si $X_1, X_2, ..., X_n$ sont des variables aléatoires indépendantes et identiquement distribuées avec une moyenne $\mu$ et une variance $\sigma^2$, alors l'estimateur de Monte-Carlo classique est donné par :

$$ \hat{\mu} = \frac{1}{n} \sum_{i=1}^n X_i $$

Avec la méthode des variables antithétiques, on génère également une suite de variables aléatoires $Y_1, Y_2, ..., Y_n$ telles que $Y_i = g(X_i)$, où $g$ est une fonction qui établit une corrélation négative entre $X$ et $Y$. L'estimateur d'intérêt est alors donné par :

$$ \hat{\mu} = \frac{1}{2n} \sum_{i=1}^n (X_i + Y_i) $$

Avec cette méthode, la variance de l'estimateur est réduite de manière significative par rapport à l'estimateur classique de Monte-Carlo, car la covariance négative entre $X$ et $Y$ permet de compenser les erreurs de chaque suite de variables aléatoires.



On souhaite calculer: $$ I= \int_0^2 e^{-x^2} dx $$

1- Utiliser Python pour calculer I directement.

In [ ]:
# méthode 1: avec scipy
from scipy.integrate import quad
import numpy as np


def f(x):
    return np.exp(-x**2)


I, erreur=quad(f, 0,2)

print('I=', I)
print('Erreur=',erreur)

I= 0.8820813907624215
Erreur= 9.793070696178202e-15


In [ ]:
# Méthode 2 :
from scipy.integrate import trapz

# Définir les points x et y pour la méthode des trapèzes
x = np.linspace(0, 2, 1000)
y = np.exp(-x**2)

# Calculer l'intégrale avec la méthode des trapèzes
trapz_result = trapz(y, x)
trapz_result


0.8820813662926712

In [ ]:
#méthode 3: intégrale de gausse avec moyenne zéro et variance 1/2

from scipy.stats import norm
from math import sqrt, pi
# norm.cdf la fonction de répartition de la loi normale
I = sqrt(pi) * (norm.cdf(2, scale=sqrt(0.5)) - 0.5) # écart type: sqrt(1/2)
I

0.8820813907624216


2- a-  Estimer I par la méthode de Monte-Carlo classique de deux façons différentes.




In [ ]:
def MC_estim(y, level=0.95):
    delta = np.mean(y)
    s2 = np.var(y) / (len(y)-1)
    e_delta = np.percentile(np.random.normal(0, 1, 10000), 100 * (1 + level) / 2) * np.sqrt(s2)
    return {'value': delta, 'var': s2, 'IC.inf': delta - e_delta, 'IC.sup': delta + e_delta, 'level': level}


In [ ]:
n = 1000

# Method 1
x = np.random.normal(scale=np.sqrt(0.5), size=n) # échantillonner la pdf
I_hat_1 = MC_estim(np.sqrt(np.pi) * ((x >= 0) & (x <= 2)))

# Method 2
u = np.random.uniform(0, 2, n)
I_hat_2 = MC_estim(2 * np.exp(-u**2))

#############################################################""
# # Définir la fonction à intégrer
# def f(x):
#     return np.exp(-x**2)

# # Estimation de l'intégrale par Monte-Carlo
# A_measure = 2  # mesure de l'intervalle [0, 2]
# I_hat = (1/n) * np.sum(f(x)) * A_measure
############################################################"""


print('estimation 1: ', I_hat_1['value'], 'de variance : ', I_hat_1['var'])
print('estimation 2: ',I_hat_2['value'], 'de variance : ', I_hat_2['var'])

# I.hat.2 est plus efficace que I.hat.1 en terme de variance


estimation 1:  0.9464903563835454 de variance :  0.0007825490313212195
estimation 2:  0.9018603771868456 de variance :  0.0004661241507182316



   b- Donner les intervalles de confiance au niveau 0.95 correspondant aux estimateurs précédents.

In [ ]:
print('Intervalle de confiance : [',I_hat_1['IC.inf'],',' , I_hat_1['IC.sup'],']')
print('Intervalle de confiance : [',I_hat_2['IC.inf'],',' , I_hat_2['IC.sup'],']')


Intervalle de confiance : [ 0.8919255511413686 , 1.0010551616257222 ]
Intervalle de confiance : [ 0.8592126959957224 , 0.9445080583779689 ]


3- a- Pour chacun des estimateurs précédents, proposer une méthode alternative utilisant une variable antithétique.


In [ ]:
from scipy.stats import pearsonr

u = np.random.uniform(0, 2, n)

correlation_coefficient, p_value = pearsonr(np.exp(-u**2), np.exp(-(2 - u)**2))
correlation_coefficient

-0.9574455378328086

In [ ]:
# Method 1
I_hat_3 = MC_estim(0.5 * np.sqrt(np.pi) * (np.abs(x) <= 2))
print('estimation 3: ',I_hat_3['value'], 'de variance : ', I_hat_3['var'])

# Method 2
I_hat_4 = MC_estim(np.exp(-u**2) + np.exp(-(2 - u)**2))
print('estimation 4: ',I_hat_4['value'], 'de variance : ', I_hat_4['var'])


estimation 3:  0.8809095639000416 de variance :  4.688803449952333e-06
estimation 4:  0.8849532068361942 de variance :  1.0374252187300425e-05


b- Donner les intervalles de confiance au niveau 0.95 correspondant aux estimateurs précédents.

In [ ]:
print('Intervalle de confiance : [',I_hat_3['IC.inf'],',' , I_hat_3['IC.sup'],']')

print('Intervalle de confiance : [',I_hat_4['IC.inf'],',' , I_hat_4['IC.sup'],']')


Intervalle de confiance : [ 0.8766721576769972 , 0.885146970123086 ]
Intervalle de confiance : [ 0.8787018003549003 , 0.891204613317488 ]
